In [2]:
import numpy as np
import pandas as pd
import os
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator


ModuleNotFoundError: No module named 'scipy'

In [ ]:
# constantes das pastas de treino e validação
train_dir = 'train_data/'
val_dir = 'val_data/'

In [ ]:
train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(shear_range=0.2, rotation_range=120, horizontal_flip=True, validation_split=0.2, rescale = 1./255,)
val_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,)

train_dataset = train_data_gen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='categorical', subset='training')
val_dataset = train_data_gen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='categorical', subset='validation')


test_dataset = val_data_gen.flow_from_directory(val_dir, target_size=(64, 64), batch_size=32, class_mode='categorical')


In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=(64, 64, 3)))
model.add(tf.keras.layers.Conv2D(filters= 128, kernel_size=(5, 5), activation='relu', strides=(1,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.1))

model.add(tf.keras.layers.Conv2D(filters= 64, kernel_size=(3, 3), activation='relu', strides=(1,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(tf.keras.layers.Conv2D(filters= 32, kernel_size=(2, 2), activation='relu', strides=(1,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 64, activation='relu'))
model.add(Dropout(0.1))
model.add(tf.keras.layers.Dense(units = 5, activation='softmax', name = 'saida'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 30752)             0         
                                                                 
 dense_16 (Dense)            (None, 64)                1968192   
                                                                 
 dense_17 (Dense)            (None, 5)                 325       
                                                                 
Total params: 1969413 (7.51 MB)
Trainable params: 1969413 (7.51 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'max', patience = 10)

history = model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks = [early_stopping])


In [ ]:
# Visualizando o ERRO de treinamento
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], ':')
plt.title('Função perda')
plt.yscale('log')
plt.ylabel('Erro')
plt.xlabel('Épocas')
plt.legend(['Erro de Treinamento', 'Erro de Validação'])
plt.grid(True)
plt.show()


In [ ]:
# Plotando o treinamento e a validação da accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'], '--')
plt.yscale("log")
plt.title('Acurácia')
plt.ylabel('Acurácia')
plt.xlabel('Épocas')
plt.legend(['Treino', 'Validação'])
plt.grid(True)
plt.show()

In [ ]:
# Previsões do modelo no conjunto de validação
val_predictions = model.predict(val_dataset)
val_pred_classes = np.argmax(val_predictions, axis=1)

# Rótulos verdadeiros do conjunto de validação
val_true_classes = val_dataset.classes

In [ ]:
# Calcular acurácia
accuracy = accuracy_score(val_true_classes, val_pred_classes)
print("Acurácia (Accuracy):", accuracy)

In [ ]:
# Calcular precisão
precision = precision_score(val_true_classes, val_pred_classes, average='macro')
print("Precisão (Precision):", precision)

In [ ]:
# Calcular recall
recall = recall_score(val_true_classes, val_pred_classes, average='macro')
print("Recall:", recall)

In [ ]:
# Calcular F1 score
#f1 = f1_score(val_true_classes, val_pred_classes, average='macro')
#print("F1 Score:", f1)

In [ ]:
# Calcular e exibir a matriz de confusão
cm = confusion_matrix(val_true_classes, val_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
class_mapping = train_dataset.class_indices
print(class_mapping)


In [ ]:
# Testando o modelo
teste_image_path = 'moeda.jpg'  # Substitua pelo caminho da sua imagem de teste
teste_image = tf.keras.preprocessing.image.load_img(teste_image_path, target_size=(64, 64))
teste_image_array = tf.keras.preprocessing.image.img_to_array(teste_image)
teste_image_array = np.expand_dims(teste_image_array, axis=0)  # Adiciona uma dimensão para representar o lote (batch)

# Normaliza a imagem
#teste_image_array /= 255.0

# Predição
saida_predita = model.predict(teste_image_array)

print('Saída Preditada:', saida_predita)

In [ ]:
# Mapeamento de números de classe para nomes de classe
class_name = {
    0: "Moeda de 5 centavos",
    1: "Moeda de 10 centavos",
    2: "Moeda de 25 centavos",
    3: "Moeda de 50 centavos",
    4: "Moeda de 1 real"
}

# Obtém o número da classe prevista
classe_prevista = np.argmax(saida_predita)

# Obtém o nome da classe prevista usando o mapeamento
nome_classe_prevista = class_name[classe_prevista]

print('Classe Prevista:', nome_classe_prevista)


In [ ]:
# Gerar previsões no conjunto de validação
saida_predida_test = model.predict(test_dataset)

# Exibir as primeiras 5 previsões
print(saida_predida_test[:5])


In [ ]:
model.save('mode_acurracy82.h5')

In [ ]:
dados = pd.DataFrame(history.history)
dados.head()

In [ ]:
dados.to_csv('loss.csv')